In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
%matplotlib inline
sns.set()

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train["source"] = 1
test["source"] =0

data = pd.concat([train, test]).reset_index(drop=True)

In [4]:
data.isna().sum()

customer_id           0
age                   0
job                   0
marital               0
education             0
default               0
housing               0
loan                  0
contact               0
month                 0
day_of_week           0
duration              0
campaign              0
pdays                 0
previous              0
poutcome              0
emp_var_rate          0
cons_price_idx        0
cons_conf_idx         0
euribor3m             0
nr_employed           0
subscribed        12357
source                0
dtype: int64

In [5]:
data.drop(["customer_id"], axis=1, inplace=True)

In [6]:
cat_cols = data.columns[data.dtypes == object]
for col in cat_cols:
    print(col.upper(), "column has", data[col].nunique(), "unique values:\n", data[col].value_counts())

JOB column has 12 unique values:
 admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64
MARITAL column has 4 unique values:
 married     24928
single      11568
divorced     4612
unknown        80
Name: marital, dtype: int64
EDUCATION column has 8 unique values:
 university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64
DEFAULT column has 3 unique values:
 no         32588
unknown     8597
yes            3
Name: default, dtype: int64
HOUSING column has 3 unique values:
 yes        21576
no         18622
unknown      990
Name: housing, dtype: in

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_cols:
    data[col] = le.fit_transform(data[col])
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,subscribed,source
0,31,0,1,6,0,0,0,0,2,1,...,999,1,0,-2.97,46.3565,-23.10,1.711,5023.5,0.0,1
1,31,9,2,6,0,0,0,1,6,0,...,999,0,1,-1.77,46.4465,-32.34,2.252,5099.1,0.0,1
2,47,1,1,1,1,2,0,1,6,2,...,999,0,1,1.13,46.9970,-25.48,5.862,5191.0,0.0,1
3,36,7,1,6,0,0,0,0,6,2,...,999,1,0,-1.77,46.4465,-32.34,2.329,5099.1,0.0,1
4,34,0,2,3,0,0,0,0,6,0,...,999,0,1,-1.77,46.4465,-32.34,2.252,5099.1,0.0,1


In [8]:
train_df = data.loc[data["source"] ==1]
test_df = data.loc[data["source"] ==0]

In [9]:
test_df.drop(["source", "subscribed"], axis=1, inplace=True)
train_df.drop(["source"], axis=1, inplace=True)

C:\Users\user\anaconda3.2020\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [10]:
y = train.subscribed
X = train_df.drop(["subscribed"], axis=1)

In [11]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 1234)

In [12]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score

model = LogisticRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(f1_score(pred, y_test))

0.4691689008042895


C:\Users\user\anaconda3.2020\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
from xgboost import XGBClassifier
xgboostmodel = XGBClassifier()

xgboostmodel.fit(X_train, y_train)
pred = xgboostmodel.predict(X_test)
print(f1_score(pred, y_test))

C:\Users\user\anaconda3.2020\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:01:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.5587734241908007


In [14]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier()

In [15]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(f1_score(pred,y_test))

Learning rate set to 0.037166
0:	learn: 0.6324502	total: 241ms	remaining: 4m
1:	learn: 0.5770869	total: 288ms	remaining: 2m 23s
2:	learn: 0.5424137	total: 326ms	remaining: 1m 48s
3:	learn: 0.5040898	total: 380ms	remaining: 1m 34s
4:	learn: 0.4687271	total: 426ms	remaining: 1m 24s
5:	learn: 0.4376031	total: 496ms	remaining: 1m 22s
6:	learn: 0.4113342	total: 563ms	remaining: 1m 19s
7:	learn: 0.3899891	total: 612ms	remaining: 1m 15s
8:	learn: 0.3691318	total: 657ms	remaining: 1m 12s
9:	learn: 0.3508040	total: 730ms	remaining: 1m 12s
10:	learn: 0.3352641	total: 778ms	remaining: 1m 9s
11:	learn: 0.3215303	total: 852ms	remaining: 1m 10s
12:	learn: 0.3068826	total: 904ms	remaining: 1m 8s
13:	learn: 0.2935509	total: 953ms	remaining: 1m 7s
14:	learn: 0.2844767	total: 1.04s	remaining: 1m 8s
15:	learn: 0.2768333	total: 1.09s	remaining: 1m 7s
16:	learn: 0.2707649	total: 1.14s	remaining: 1m 5s
17:	learn: 0.2632808	total: 1.21s	remaining: 1m 6s
18:	learn: 0.2581885	total: 1.28s	remaining: 1m 5s
19:	

165:	learn: 0.1652178	total: 11.4s	remaining: 57.1s
166:	learn: 0.1650600	total: 11.5s	remaining: 57.2s
167:	learn: 0.1649503	total: 11.5s	remaining: 57s
168:	learn: 0.1648340	total: 11.6s	remaining: 56.9s
169:	learn: 0.1647450	total: 11.6s	remaining: 56.8s
170:	learn: 0.1647215	total: 11.7s	remaining: 56.7s
171:	learn: 0.1645993	total: 11.7s	remaining: 56.5s
172:	learn: 0.1644956	total: 11.8s	remaining: 56.4s
173:	learn: 0.1643861	total: 11.9s	remaining: 56.5s
174:	learn: 0.1642886	total: 12s	remaining: 56.4s
175:	learn: 0.1641981	total: 12s	remaining: 56.2s
176:	learn: 0.1641493	total: 12.1s	remaining: 56.1s
177:	learn: 0.1640966	total: 12.1s	remaining: 56s
178:	learn: 0.1639766	total: 12.2s	remaining: 55.9s
179:	learn: 0.1638756	total: 12.2s	remaining: 55.7s
180:	learn: 0.1638080	total: 12.3s	remaining: 55.7s
181:	learn: 0.1637537	total: 12.4s	remaining: 55.5s
182:	learn: 0.1636139	total: 12.4s	remaining: 55.4s
183:	learn: 0.1635483	total: 12.5s	remaining: 55.4s
184:	learn: 0.163439

325:	learn: 0.1518609	total: 20.9s	remaining: 43.2s
326:	learn: 0.1518128	total: 21s	remaining: 43.2s
327:	learn: 0.1517124	total: 21.1s	remaining: 43.2s
328:	learn: 0.1515664	total: 21.2s	remaining: 43.2s
329:	learn: 0.1514901	total: 21.3s	remaining: 43.3s
330:	learn: 0.1513662	total: 21.4s	remaining: 43.2s
331:	learn: 0.1512998	total: 21.4s	remaining: 43.1s
332:	learn: 0.1512454	total: 21.5s	remaining: 43s
333:	learn: 0.1512008	total: 21.5s	remaining: 42.9s
334:	learn: 0.1511793	total: 21.6s	remaining: 42.8s
335:	learn: 0.1510920	total: 21.7s	remaining: 42.8s
336:	learn: 0.1510766	total: 21.8s	remaining: 42.8s
337:	learn: 0.1510167	total: 21.8s	remaining: 42.8s
338:	learn: 0.1509437	total: 21.9s	remaining: 42.7s
339:	learn: 0.1508419	total: 22s	remaining: 42.7s
340:	learn: 0.1508005	total: 22.1s	remaining: 42.6s
341:	learn: 0.1507248	total: 22.1s	remaining: 42.6s
342:	learn: 0.1505933	total: 22.2s	remaining: 42.6s
343:	learn: 0.1505181	total: 22.3s	remaining: 42.5s
344:	learn: 0.1504

486:	learn: 0.1417808	total: 31.8s	remaining: 33.5s
487:	learn: 0.1417146	total: 31.9s	remaining: 33.5s
488:	learn: 0.1416295	total: 32s	remaining: 33.5s
489:	learn: 0.1415774	total: 32.1s	remaining: 33.4s
490:	learn: 0.1414994	total: 32.1s	remaining: 33.3s
491:	learn: 0.1414531	total: 32.2s	remaining: 33.2s
492:	learn: 0.1414349	total: 32.2s	remaining: 33.2s
493:	learn: 0.1413236	total: 32.3s	remaining: 33.1s
494:	learn: 0.1412681	total: 32.4s	remaining: 33s
495:	learn: 0.1412007	total: 32.4s	remaining: 32.9s
496:	learn: 0.1411721	total: 32.5s	remaining: 32.9s
497:	learn: 0.1411261	total: 32.5s	remaining: 32.8s
498:	learn: 0.1409947	total: 32.6s	remaining: 32.7s
499:	learn: 0.1409130	total: 32.6s	remaining: 32.6s
500:	learn: 0.1408615	total: 32.7s	remaining: 32.5s
501:	learn: 0.1408427	total: 32.7s	remaining: 32.5s
502:	learn: 0.1407990	total: 32.8s	remaining: 32.4s
503:	learn: 0.1407344	total: 32.8s	remaining: 32.3s
504:	learn: 0.1406790	total: 32.9s	remaining: 32.3s
505:	learn: 0.14

647:	learn: 0.1323651	total: 42.6s	remaining: 23.1s
648:	learn: 0.1323575	total: 42.6s	remaining: 23.1s
649:	learn: 0.1323055	total: 42.7s	remaining: 23s
650:	learn: 0.1322649	total: 42.7s	remaining: 22.9s
651:	learn: 0.1322316	total: 42.8s	remaining: 22.8s
652:	learn: 0.1322035	total: 42.9s	remaining: 22.8s
653:	learn: 0.1321557	total: 42.9s	remaining: 22.7s
654:	learn: 0.1321006	total: 42.9s	remaining: 22.6s
655:	learn: 0.1320383	total: 43s	remaining: 22.5s
656:	learn: 0.1319974	total: 43s	remaining: 22.5s
657:	learn: 0.1319459	total: 43.1s	remaining: 22.4s
658:	learn: 0.1318798	total: 43.1s	remaining: 22.3s
659:	learn: 0.1318050	total: 43.2s	remaining: 22.2s
660:	learn: 0.1317581	total: 43.3s	remaining: 22.2s
661:	learn: 0.1316812	total: 43.3s	remaining: 22.1s
662:	learn: 0.1316233	total: 43.3s	remaining: 22s
663:	learn: 0.1315857	total: 43.4s	remaining: 22s
664:	learn: 0.1314911	total: 43.5s	remaining: 21.9s
665:	learn: 0.1314683	total: 43.5s	remaining: 21.8s
666:	learn: 0.1313881	

808:	learn: 0.1237100	total: 51.7s	remaining: 12.2s
809:	learn: 0.1236498	total: 51.7s	remaining: 12.1s
810:	learn: 0.1236187	total: 51.8s	remaining: 12.1s
811:	learn: 0.1235439	total: 51.9s	remaining: 12s
812:	learn: 0.1234854	total: 52s	remaining: 12s
813:	learn: 0.1234618	total: 52.1s	remaining: 11.9s
814:	learn: 0.1233848	total: 52.2s	remaining: 11.8s
815:	learn: 0.1232941	total: 52.3s	remaining: 11.8s
816:	learn: 0.1232152	total: 52.4s	remaining: 11.7s
817:	learn: 0.1231588	total: 52.4s	remaining: 11.7s
818:	learn: 0.1230960	total: 52.5s	remaining: 11.6s
819:	learn: 0.1230543	total: 52.6s	remaining: 11.5s
820:	learn: 0.1230201	total: 52.6s	remaining: 11.5s
821:	learn: 0.1229697	total: 52.7s	remaining: 11.4s
822:	learn: 0.1229083	total: 52.8s	remaining: 11.4s
823:	learn: 0.1228657	total: 52.9s	remaining: 11.3s
824:	learn: 0.1228024	total: 53s	remaining: 11.2s
825:	learn: 0.1227491	total: 53s	remaining: 11.2s
826:	learn: 0.1227114	total: 53.1s	remaining: 11.1s
827:	learn: 0.1226800	

971:	learn: 0.1163506	total: 1m	remaining: 1.75s
972:	learn: 0.1163038	total: 1m	remaining: 1.68s
973:	learn: 0.1162315	total: 1m	remaining: 1.62s
974:	learn: 0.1161890	total: 1m	remaining: 1.56s
975:	learn: 0.1161458	total: 1m	remaining: 1.49s
976:	learn: 0.1161104	total: 1m	remaining: 1.43s
977:	learn: 0.1160601	total: 1m	remaining: 1.37s
978:	learn: 0.1160142	total: 1m	remaining: 1.3s
979:	learn: 0.1159324	total: 1m	remaining: 1.24s
980:	learn: 0.1158854	total: 1m	remaining: 1.18s
981:	learn: 0.1158545	total: 1m	remaining: 1.12s
982:	learn: 0.1158103	total: 1m 1s	remaining: 1.05s
983:	learn: 0.1157677	total: 1m 1s	remaining: 993ms
984:	learn: 0.1157374	total: 1m 1s	remaining: 931ms
985:	learn: 0.1157131	total: 1m 1s	remaining: 868ms
986:	learn: 0.1156849	total: 1m 1s	remaining: 806ms
987:	learn: 0.1156599	total: 1m 1s	remaining: 744ms
988:	learn: 0.1156176	total: 1m 1s	remaining: 682ms
989:	learn: 0.1155733	total: 1m 1s	remaining: 619ms
990:	learn: 0.1155446	total: 1m 1s	remaining: 

In [16]:
predictions = xgboostmodel.predict(test_df)

In [17]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [18]:
sub = pd.read_csv('sample_submission.csv')

In [19]:
sub.head()

,customer_id,subscribed
0,customer_id_32884,1
1,customer_id_3169,1
2,customer_id_32206,1
3,customer_id_9403,1
4,customer_id_14020,1


In [20]:
predtt = pd.DataFrame({"customer_id": sub["customer_id"],"subscribed":predictions})

In [21]:
predtt.to_csv('ayopredis.csv', index = False)